In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy.special import softmax
import matplotlib.pyplot as plt
import os
import sklearn.metrics as metrics
from scipy import stats
import seaborn as sns
from utils import *


# Post-process model output 

In [2]:

df = None
healthy_df = None
acts = ['brushing','combing','deodorant','drinking','facewash','feeding','glasses','shelf','RTT']


primatives = {0:'idle', 1: 'reach', 2: 'reposition', 3: 'stabilize', 4: 'transport'}

n_folds = 5    
for i in range(1,n_folds+1):
    data_dict = unpickling(f'/gpfs/scratch/by2026/asrf_hc/result/sensors_all/in_channel-77_param_search-True_dataset-sensors_all_split-{i}/predictions_loss_model._stroke.p')
    data_dict['sub'] = []
    data_dict['activity'] = []
    data_dict['rep'] = []
    for i in range(5):
        data_dict[f'prob_cal_{i}'] = []

    for j in range(len(data_dict['name'])):
        data_dict['probs'][j] = softmax(data_dict['probs'][j][0],axis = 0)
        data_dict['sub'].append(data_dict['name'][j].split('_')[0])
        for a in acts:
            if a=='deodorant':
                if 'deodrant' in data_dict['name'][j] or 'deodorant' in data_dict['name'][j]:
                    data_dict['activity'].append(a)
            elif a=='facewash':
                if 'face' in data_dict['name'][j]:
                    data_dict['activity'].append(a)
            else:
                if a in data_dict['name'][j]:
                    data_dict['activity'].append(a)
        data_dict['rep'].append(data_dict['name'][j][-1])
        data_dict['gt'][j] = data_dict['gt'][j].numpy() 
        data_dict['prob_cal_0'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==0])
        data_dict['prob_cal_1'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==1])
        data_dict['prob_cal_2'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==2]) 
        data_dict['prob_cal_3'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==3])
        data_dict['prob_cal_4'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==4])
        
    temp = pd.DataFrame({'sub':data_dict['sub'],'act':data_dict['activity'],'rep':data_dict['rep'],'gt':data_dict['gt'],'pred':data_dict['r_pred'],
                                 'probs':data_dict['probs'],
                                 'prob_cal_0':data_dict['prob_cal_0'], 'prob_cal_1':data_dict['prob_cal_1'], 'prob_cal_2':data_dict['prob_cal_2'], 'prob_cal_3':data_dict['prob_cal_3'], 'prob_cal_4':data_dict['prob_cal_4']
                                })
    temp['seq_prob'] = temp.probs.map(collate_confidence)
    temp['mean_seq_prob'] = temp.seq_prob.map(lambda x:np.mean(x))
    temp['mean_motion_seq_prob'] = temp[['prob_cal_1','prob_cal_2', 'prob_cal_4']].apply(lambda x: np.mean(np.hstack(x)),axis=1)
    temp['mean_non_motion_seq_prob'] = temp[['prob_cal_0','prob_cal_3']].apply(lambda x: np.mean(np.hstack(x)),axis=1)
            
    if df is None:
        df=temp
    else:
        df = pd.concat([df,temp],ignore_index=True)
        
    data_dict = unpickling(f'/gpfs/scratch/by2026/asrf_hc/result/sensors_all/in_channel-77_param_search-True_dataset-sensors_all_split-{i}/predictions_loss_model.p')
    data_dict['sub'] = []
    data_dict['activity'] = []
    data_dict['rep'] = []
    for i in range(5):
        data_dict[f'prob_cal_{i}'] = []
        
    for j in range(len(data_dict['name'])):
        data_dict['probs'][j] = softmax(data_dict['probs'][j][0],axis = 0)
        data_dict['sub'].append(data_dict['name'][j].split('_')[0])
        for a in acts:
            if a=='deodorant':
                if 'deodrant' in data_dict['name'][j] or 'deodorant' in data_dict['name'][j]:
                    data_dict['activity'].append(a)
            elif a=='facewash':
                if 'face' in data_dict['name'][j]:
                    data_dict['activity'].append(a)
            else:
                if a in data_dict['name'][j]:
                    data_dict['activity'].append(a)
        data_dict['rep'].append(data_dict['name'][j][-1])
        data_dict['gt'][j] = data_dict['gt'][j].numpy() 
        data_dict['prob_cal_0'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==0])
        data_dict['prob_cal_1'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==1])
        data_dict['prob_cal_2'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==2]) 
        data_dict['prob_cal_3'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==3])
        data_dict['prob_cal_4'].append(data_dict['probs'][j].max(axis=0)[data_dict['probs'][j].argmax(axis=0)==4])

                
    temp = pd.DataFrame({'sub':data_dict['sub'],'act':data_dict['activity'],'rep':data_dict['rep'],'gt':data_dict['gt'],'pred':data_dict['r_pred'],
                                 'probs':data_dict['probs'],
                                 'prob_cal_0':data_dict['prob_cal_0'], 'prob_cal_1':data_dict['prob_cal_1'], 'prob_cal_2':data_dict['prob_cal_2'], 'prob_cal_3':data_dict['prob_cal_3'], 'prob_cal_4':data_dict['prob_cal_4']
                                })
    temp['seq_prob'] = temp.probs.map(collate_confidence)
    temp['mean_seq_prob'] = temp.seq_prob.map(lambda x:np.mean(x))
    temp['mean_motion_seq_prob'] = temp[['prob_cal_1','prob_cal_2', 'prob_cal_4']].apply(lambda x: np.mean(np.hstack(x)),axis=1)
    temp['mean_non_motion_seq_prob'] = temp[['prob_cal_0','prob_cal_3']].apply(lambda x: np.mean(np.hstack(x)),axis=1)
               
    if healthy_df is None:
        healthy_df = temp
    else:
        healthy_df = pd.concat([healthy_df,temp],ignore_index=True)

        


/gpfs/share/apps/anaconda3/gpu/5.2.0/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/gpfs/share/apps/anaconda3/gpu/5.2.0/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [4]:
df.to_pickle('stroke_raw.pkl')
healthy_df.to_pickle('healthy_raw.pkl')